In [4]:
# requirements 
%pip install pandas --quiet 


You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# imports
print("Starting imports...")
import pandas as pd


print("Successfully imported!")